# 0、导入相关lib

In [1]:
import numpy as np
import pandas as pd

# 1、生成测试小规模数据mini_data

In [3]:
data = pd.read_table('./demo.txt', sep=',', names=['id','lat','lng','status','datetime'])
data.info

<bound method DataFrame.info of              id        lat         lng  status           datetime
0             1  30.624806  104.136604       1  2014/8/3 21:18:46
1             1  30.624809  104.136612       1  2014/8/3 21:18:15
2             1  30.624811  104.136587       1  2014/8/3 21:20:17
3             1  30.624811  104.136596       1  2014/8/3 21:19:16
4             1  30.624811  104.136619       1  2014/8/3 21:17:44
...         ...        ...         ...     ...                ...
53045402  13605  30.732042  103.998304       1  2014/8/3 11:24:41
53045403  13605  30.732262  103.997677       1  2014/8/3 11:28:58
53045404  13605  30.732322  103.997589       1  2014/8/3 11:25:21
53045405  13605  30.732406  103.997811       1  2014/8/3 11:25:12
53045406  13605  30.732416  103.997727       1  2014/8/3 11:25:15

[53045407 rows x 5 columns]>

In [8]:
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x:'%.3f'%x) # 小数点后面保留3位小数，诸如此类，按需修改吧
data.describe()

,id,lat,lng,status
count,53045407.000,53045407.000,53045407.000,53045407.000
mean,6878.455,30.652,104.060,0.564
std,3907.969,0.043,0.052,0.496
min,1.000,30.291,103.270,0.000
25%,3545.000,30.629,104.032,0.000
50%,6931.000,30.652,104.064,1.000
75%,10248.000,30.677,104.092,1.000
max,13605.000,31.032,104.610,1.000


In [30]:
# 选取点最多的二十条轨迹
# len(data['id'].value_counts()) # 13605
id_data = data['id'].value_counts().sort_values(ascending=False)
# id_data.to_csv('./demo_id.csv')
test_data_id = id_data[0:20].index.to_list()

In [32]:
mini_data = data[data['id'].isin(test_data_id)]
mini_data.to_csv('./mini_data.csv', index=False)
mini_data

,id,lat,lng,status,datetime
821028,216,30.581143,103.935058,0,2014/8/3 12:33:53
821029,216,30.581333,103.935600,0,2014/8/3 12:33:43
821030,216,30.581335,103.935605,0,2014/8/3 12:33:32
821031,216,30.581436,103.935751,0,2014/8/3 12:33:22
821032,216,30.581954,103.934489,0,2014/8/3 12:34:03
...,...,...,...,...,...
52712185,13521,30.705482,104.066774,0,2014/8/3 23:35:06
52712186,13521,30.705484,104.066774,0,2014/8/3 23:34:46
52712187,13521,30.705510,104.066774,0,2014/8/3 23:34:36
52712188,13521,30.705516,104.066759,0,2014/8/3 23:34:56


In [10]:
data = pd.read_table('./demo.txt', sep=',', names=['id','lat','lng','status','datetime'])
mini_data = data[data['id'] < 100]

In [15]:
mini_data.describe()
mini_data.to_csv('./dataset/mini_data.csv', index=False)

In [14]:
# read mini_data
mini_data['id'].value_counts().sort_values(ascending=False)

16    6457
92    6341
75    6334
54    6278
50    6222
      ... 
79    1028
48     966
63     697
64     430
23     393
Name: id, Length: 99, dtype: int64

# 2、空间网格划分


In [2]:
# 导入数据
data = pd.read_csv('./dataset/mini_data.csv')
data.describe()

,id,lat,lng,status
count,359793.000000,359793.000000,359793.000000,359793.000000
mean,52.747163,30.654660,104.058381,0.545761
std,28.203944,0.047194,0.060309,0.497902
min,1.000000,30.322692,103.497147,0.000000
25%,31.000000,30.629347,104.032637,0.000000
50%,55.000000,30.653516,104.066214,1.000000
75%,76.000000,30.678354,104.093654,1.000000
max,99.000000,30.995640,104.275602,1.000000


In [23]:
# 生成空间网格

LAT1 = 30.322692
LAT2 = 30.995640

LNG1 = 103.497147
LNG2 = 104.275602

column_num, row_num = 100, 100

def generateGirdID(lat,lng):
    # 若在范围外的点，返回-1
    if lng < LNG1 or lng > LNG2 or lat < LAT1 or lat > LAT2:
        return -1
    # 把经度范围根据列数等分切割
    column = (LNG2 - LNG1)/column_num
    # 把纬度范围根据行数数等分切割
    row = (LAT2 - LAT1)/row_num
    # 二维矩阵坐标索引转换为一维ID，即： （列坐标区域（向下取整）+ 1） + （行坐标区域 * 列数）
    return int((lng-LNG1)/column)+ 1 + int((lat-LAT1)/row) * column_num

In [24]:
data['gird_id'] = data.apply(lambda x: generateGirdID(x['lat'], x['lng']), axis=1)
data.describe()

,id,lat,lng,status,gird_id
count,359793.000000,359793.000000,359793.000000,359793.000000,359793.000000
mean,52.747163,30.654660,104.058381,0.545761,4955.589333
std,28.203944,0.047194,0.060309,0.497902,700.144836
min,1.000000,30.322692,103.497147,0.000000,59.000000
25%,31.000000,30.629347,104.032637,0.000000,4581.000000
50%,55.000000,30.653516,104.066214,1.000000,4973.000000
75%,76.000000,30.678354,104.093654,1.000000,5279.000000
max,99.000000,30.995640,104.275602,1.000000,10016.000000


In [21]:
data['gird_id'].value_counts().sort_values(ascending=False)

3760    4986
4974    4716
4981    3634
4975    3510
5674    3375
        ... 
6759       1
7168       1
6920       1
5346       1
4286       1
Name: gird_id, Length: 1400, dtype: int64